Pelo que foi visto durante a análise dos datasets (arquivo `data_analysis.ipynb`), percebe-se que os dados que mais relevantes para o projeto são os que contém os **itens complementares**, ou seja: precisamos lidar os dados das notícias, a fim de ter algo bom o suficiente para começarmos a desenvolver o modelo final.

Separamos, então, em duas etapas: a engenharia de features (neste notebook) e o pré-processamento dos dados, que servirá para treinar o modelo de machine learning.

## Setup

In [1]:
from pathlib import Path

import numpy as np
import pandas as pd
import pyarrow.csv as pv

In [2]:
pd.options.future.infer_string = True  # type: ignore[attr-defined]

In [3]:
def read_csv_pyarrow(path: Path) -> pd.DataFrame:
    """Read a CSV file with PyArrow as backend.

    Args:
        path (Path): the CSV file path

    Returns:
        pd.DataFrame: a Pandas DataFrame

    """
    try:
        return pd.read_csv(
            path,
            engine="pyarrow",
            dtype_backend="pyarrow",
        )
    except pd.errors.ParserError:
        return pv.read_csv(
            path,
            parse_options=pv.ParseOptions(newlines_in_values=True),
        ).to_pandas(types_mapper=pd.ArrowDtype)

In [4]:
itens_path = "../data/raw/itens/itens/*.csv"
itens_files = Path.cwd().glob(itens_path)

itens = pd.concat(
    (read_csv_pyarrow(file) for file in itens_files),
    ignore_index=True,
)

In [5]:
itens.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 255603 entries, 0 to 255602
Data columns (total 7 columns):
 #   Column    Non-Null Count   Dtype                        
---  ------    --------------   -----                        
 0   page      255603 non-null  string[pyarrow]              
 1   url       255603 non-null  string[pyarrow]              
 2   issued    255603 non-null  timestamp[s, tz=UTC][pyarrow]
 3   modified  255603 non-null  timestamp[s, tz=UTC][pyarrow]
 4   title     255603 non-null  string[pyarrow]              
 5   body      255603 non-null  string[pyarrow]              
 6   caption   255603 non-null  string[pyarrow]              
dtypes: string[pyarrow](5), timestamp[s, tz=UTC][pyarrow](2)
memory usage: 880.0 MB


In [6]:
itens.head(2)

,page,url,issued,modified,title,body,caption
0,13db0ab1-eea2-4603-84c4-f40a876c7400,http://g1.globo.com/am/amazonas/noticia/2022/0...,2022-06-18 20:37:45+00:00,2023-04-15 00:02:08+00:00,Caso Bruno e Dom: 3º suspeito tem prisão tempo...,"Após audiência de custódia, a Justiça do Amazo...",Jeferson da Silva Lima foi escoltado por agent...
1,92907b73-5cd3-4184-8d8c-e206aed2bf1c,http://g1.globo.com/pa/santarem-regiao/noticia...,2019-06-20 17:19:52+00:00,2023-06-16 20:19:15+00:00,Linguajar dos santarenos é diferenciado e chei...,Vista aérea de Santarém Ádrio Denner/ AD Produ...,As expressões santarenas não significam apenas...


## Gerando lista de tags para as notícias

In [7]:
itens.iloc[0].url

'http://g1.globo.com/am/amazonas/noticia/2022/06/18/caso-bruno-e-dom-3o-suspeito-tem-prisao-temporaria-decretada-pela-justica-do-am.ghtml'

Conforme dito na análise dos dados, a coluna `url` é rica em informações sobre as notícias, sendo que a maioria das informações já aparece em outras colunas da tabela, como a data de publicação e o título da notícia, porém uma das informações que não temos são as **tags**.

A grande maioria das URLs do G1 seguem a seguinte estrutura:

- \<URL do site><Tags da notícia><Data de publicação><Título da notícia>

Exemplo:\
**http://g1.globo.com/am/amazonas/noticia/2022/06/18/caso-bruno-e-dom-3o-suspeito-tem-prisao-temporaria-decretada-pela-justica-do-am.ghtml**

- URL base do G1: **http://g1.globo.com/**
- Tags da notícia: **am/amazonas/noticia/**
- Data de publicação: **2022/06/18/**
- Título da notícia: **caso-bruno-e-dom-3o-suspeito-tem-prisao-temporaria-decretada-pela-justica-do-am.ghtml**

Sendo assim, através de um tratamento de strings, pode-se gerar tags para a notícia.\
No exemplo acima, a notícia possui as tags "AM", "Amazonas" e "Notícia". A tag "noticia" indica que essa página é uma notícia - pela análise, sabe-se que nem todas são; Já a tag "AM", indica a UF; enquanto "Amazonas", o lugar mais exato onde ocorreu a notícia.

O trabalho aqui será generalizar o tratamento das URLs do G1, a fim de gerar tags que façam sentido para nosso projeto. Sendo assim, pensamos em algumas coisas:

- Quando um ano aparecer nas tags, a tag gerada deve ser a soma da tag anterior e o ano. Ex.:
  - Na URL (...)/sp/sao-paulo/eleicoes/2022/noticia/2022/05/04/, temos "eleicoes" seguido de "2022", então a tag formada deve ser "eleicoes-2022"
  - O motivo disso é simples: o ano possui relação com a palavra anterior (eleicoes), logo, não faria sentido criar uma tag contendo apenas "2022"
- Quando temos estado e cidade na URL, não podemos deixar a cidade sozinha (já que muitas possuem um nome repetido), logo, similar ao item acima, vamos juntar as duas: sp/sao-paulo viraria sp-sao-paulo.
  - Obs.: No momento, não faremos o mesmo para estados, pois um usuário pode gostar de ver notícias de SP no geral, então precisamos gerar uma tag para a UF (sp) e outra para a cidade (sp-sao-paulo)
- Por mais que a base seja quase 100% composta por notícias, sabemos que nem todas as páginas são uma, logo, não podemos desconsiderar esta tag

In [8]:
import re

Primeiramente, precisamos tratar as URLs. Nem todas estão completas, sendo apenas "http://especiais.g1.globo.com". Isso não nos trará nenhum valor, já que pela URL não sabemos do que a notícia se trata.

Fazendo a análise abaixo, percebe-se que, nesses casos, a coluna "page" acaba contendo a URL correta da notícia. Sendo assim, iremos ajustar essas URLs para obter a mesma URL contida na coluna page.

In [9]:
itens[~itens["url"].str.contains("http://g1.globo.com")].sort_values(by="page")

,page,url,issued,modified,title,body,caption
208305,esid:conteudo_editorial_g1#materia#http://espe...,http://especiais.g1.globo.com/,2017-01-01 07:00:00+00:00,2020-01-01 12:56:22+00:00,As promessas de Marcus e Socorro,Marcus e Socorro fizeram promessas específicas...,Marcus e Socorro fizeram promessas específicas...
63936,esid:conteudo_editorial_g1#materia#http://espe...,http://especiais.g1.globo.com/,2015-12-29 07:00:00+00:00,2018-12-28 19:42:23+00:00,As promessas de Renan,Renan fez promessas específicas em um programa...,Renan fez promessas específicas em um programa...
228568,esid:conteudo_editorial_g1#materia#http://espe...,http://especiais.g1.globo.com/,2017-01-01 07:00:00+00:00,2020-01-01 12:52:31+00:00,As promessas de Rui Palmeira,Rui Palmeira fez promessas específicas em um p...,Rui Palmeira fez promessas específicas em um p...
221112,esid:conteudo_editorial_g1#materia#http://espe...,http://especiais.g1.globo.com/,2015-12-29 07:00:00+00:00,2019-01-02 12:52:53+00:00,As promessas de Waldez,Waldez fez promessas específicas em um program...,Waldez fez promessas específicas em um program...
160117,esid:conteudo_editorial_g1#materia#http://espe...,http://especiais.g1.globo.com/,2017-01-01 07:00:00+00:00,2020-01-02 16:03:47+00:00,As promessas de Clécio,Clécio fez promessas específicas em um program...,Clécio fez promessas específicas em um program...
...,...,...,...,...,...,...,...
204476,esid:conteudo_editorial_g1#materia#https://esp...,http://especiais.g1.globo.com/sp/vale-do-parai...,2018-10-07 20:00:00+00:00,2018-10-07 20:00:00+00:00,Eleições 2018 no G1 - Apuração por zona eleito...,Mapa de apuração por zona eleitoral do município,Mapa de apuração por zona eleitoral do município
204475,esid:conteudo_editorial_g1#materia#https://esp...,http://especiais.g1.globo.com/sp/vale-do-parai...,2018-10-07 20:00:00+00:00,2018-10-07 20:00:00+00:00,Eleições 2018 no G1 - Apuração por zona eleito...,Mapa de apuração por zona eleitoral do município,Mapa de apuração por zona eleitoral do município
204207,esid:conteudo_editorial_g1#materia#https://esp...,http://especiais.g1.globo.com/to/tocantins/ele...,2018-10-07 20:00:00+00:00,2018-10-07 20:00:00+00:00,"Eleições 2018 no G1 - Pesquisas, Apuração de V...",Veja os resultados da eleição em cada um dos e...,Veja os resultados da eleição em cada um dos e...
204320,esid:conteudo_editorial_g1#materia#https://esp...,http://especiais.g1.globo.com/to/tocantins/ele...,2018-10-07 20:00:00+00:00,2018-10-07 20:00:00+00:00,Eleições 2018 no G1 - Apuração por zona eleito...,Mapa de apuração por zona eleitoral do município,Mapa de apuração por zona eleitoral do município


In [10]:
def fix_page(row: pd.Series) -> pd.Series:
    """Fix the page url."""
    if row["url"] == "http://especiais.g1.globo.com/":
        match = re.search(r"http://especiais\.g1\.globo\.com/[^ ]+", row["page"])
        if match:
            row["url"] = match.group(0)
    return row


itens = itens.apply(fix_page, axis=1)
itens[~itens["url"].str.contains("http://g1.globo.com")].sort_values(by="page")

,page,url,issued,modified,title,body,caption
208305,esid:conteudo_editorial_g1#materia#http://espe...,http://especiais.g1.globo.com/acre/2017/as-pro...,2017-01-01 07:00:00+00:00,2020-01-01 12:56:22+00:00,As promessas de Marcus e Socorro,Marcus e Socorro fizeram promessas específicas...,Marcus e Socorro fizeram promessas específicas...
63936,esid:conteudo_editorial_g1#materia#http://espe...,http://especiais.g1.globo.com/alagoas/2015/as-...,2015-12-29 07:00:00+00:00,2018-12-28 19:42:23+00:00,As promessas de Renan,Renan fez promessas específicas em um programa...,Renan fez promessas específicas em um programa...
228568,esid:conteudo_editorial_g1#materia#http://espe...,http://especiais.g1.globo.com/alagoas/2017/as-...,2017-01-01 07:00:00+00:00,2020-01-01 12:52:31+00:00,As promessas de Rui Palmeira,Rui Palmeira fez promessas específicas em um p...,Rui Palmeira fez promessas específicas em um p...
221112,esid:conteudo_editorial_g1#materia#http://espe...,http://especiais.g1.globo.com/amapa/2015/as-pr...,2015-12-29 07:00:00+00:00,2019-01-02 12:52:53+00:00,As promessas de Waldez,Waldez fez promessas específicas em um program...,Waldez fez promessas específicas em um program...
160117,esid:conteudo_editorial_g1#materia#http://espe...,http://especiais.g1.globo.com/amapa/2017/as-pr...,2017-01-01 07:00:00+00:00,2020-01-02 16:03:47+00:00,As promessas de Clécio,Clécio fez promessas específicas em um program...,Clécio fez promessas específicas em um program...
...,...,...,...,...,...,...,...
204476,esid:conteudo_editorial_g1#materia#https://esp...,http://especiais.g1.globo.com/sp/vale-do-parai...,2018-10-07 20:00:00+00:00,2018-10-07 20:00:00+00:00,Eleições 2018 no G1 - Apuração por zona eleito...,Mapa de apuração por zona eleitoral do município,Mapa de apuração por zona eleitoral do município
204475,esid:conteudo_editorial_g1#materia#https://esp...,http://especiais.g1.globo.com/sp/vale-do-parai...,2018-10-07 20:00:00+00:00,2018-10-07 20:00:00+00:00,Eleições 2018 no G1 - Apuração por zona eleito...,Mapa de apuração por zona eleitoral do município,Mapa de apuração por zona eleitoral do município
204207,esid:conteudo_editorial_g1#materia#https://esp...,http://especiais.g1.globo.com/to/tocantins/ele...,2018-10-07 20:00:00+00:00,2018-10-07 20:00:00+00:00,"Eleições 2018 no G1 - Pesquisas, Apuração de V...",Veja os resultados da eleição em cada um dos e...,Veja os resultados da eleição em cada um dos e...
204320,esid:conteudo_editorial_g1#materia#https://esp...,http://especiais.g1.globo.com/to/tocantins/ele...,2018-10-07 20:00:00+00:00,2018-10-07 20:00:00+00:00,Eleições 2018 no G1 - Apuração por zona eleito...,Mapa de apuração por zona eleitoral do município,Mapa de apuração por zona eleitoral do município


In [11]:
UFS = {
    "ac",
    "al",
    "am",
    "ap",
    "ba",
    "ce",
    "df",
    "es",
    "go",
    "ma",
    "mt",
    "ms",
    "mg",
    "pa",
    "pb",
    "pr",
    "pe",
    "pi",
    "rj",
    "rn",
    "rs",
    "ro",
    "rr",
    "sc",
    "sp",
    "se",
    "to",
}


def extract_tags(url: str) -> list[str]:
    """Extrai tags de uma URL."""
    match = re.search(r"g1\.globo\.com/(.*?)/(\d{4})/\d{2}/\d{2}/", url)
    if not match:
        return []

    tags_part = match.group(1)
    tags = tags_part.split("/")

    final_tags = []
    i = 0
    while i < len(tags):
        tag = tags[i]

        # the next tag after an uf is probably a city
        # we need the +2 because some URLs end before a city name
        if tag in UFS and i + 2 < len(tags):
            # add UF to the tags
            uf = tag
            city = tags[i + 1]

            final_tags.append(uf)

            tag = f"{uf}-{city}"
            i += 1

        # the next tag is a year (four digits)
        elif i + 1 < len(tags) and re.fullmatch(r"\d{4}", tags[i + 1]):
            year = tags[i + 1]
            tag = f"{tag}-{year}"
            i += 1

        final_tags.append(tag)
        i += 1

    return final_tags

In [12]:
f"Todas as URLs contém 'g1.globo.com'? {all(itens['url'].str.contains('g1.globo.com'))}"

"Todas as URLs contém 'g1.globo.com'? True"

In [13]:
with pd.option_context("display.max_colwidth", None):
    display(itens.head().url)

0                            http://g1.globo.com/am/amazonas/noticia/2022/06/18/caso-bruno-e-dom-3o-suspeito-tem-prisao-temporaria-decretada-pela-justica-do-am.ghtml
1    http://g1.globo.com/pa/santarem-regiao/noticia/2019/06/20/linguajar-dos-santarenos-e-diferenciado-e-cheio-de-identidade-egua-tu-nao-vai-ser-leso-de-perder.ghtml
2                                                     http://g1.globo.com/mundo/noticia/2022/07/08/ex-premie-shinzo-abe-morre-apos-ser-baleado-no-japao-diz-nhk.ghtml
3                          http://g1.globo.com/politica/noticia/2021/09/09/relator-no-stf-fachin-vota-contra-marco-temporal-para-demarcacao-de-terras-indigenas.ghtml
4                  http://g1.globo.com/politica/noticia/2021/09/15/apos-2-votos-pedido-de-vista-suspende-julgamento-no-stf-sobre-demarcacao-de-terras-indigenas.ghtml
Name: url, dtype: string

Testando abaixo, vimos que as tags geradas ficaram exatamente no formato que queremos, então agora podemos aplicá-la em todo o dataset

In [14]:
itens["url"].head().apply(extract_tags)

0           [am, am-amazonas, noticia]
1    [pa, pa-santarem-regiao, noticia]
2                     [mundo, noticia]
3                  [politica, noticia]
4                  [politica, noticia]
Name: url, dtype: object

In [15]:
itens["tags"] = itens["url"].apply(extract_tags)

In [16]:
itens[["url", "tags"]]

,url,tags
0,http://g1.globo.com/am/amazonas/noticia/2022/0...,"[am, am-amazonas, noticia]"
1,http://g1.globo.com/pa/santarem-regiao/noticia...,"[pa, pa-santarem-regiao, noticia]"
2,http://g1.globo.com/mundo/noticia/2022/07/08/e...,"[mundo, noticia]"
3,http://g1.globo.com/politica/noticia/2021/09/0...,"[politica, noticia]"
4,http://g1.globo.com/politica/noticia/2021/09/1...,"[politica, noticia]"
...,...,...
255598,http://g1.globo.com/politica/eleicoes/2022/not...,"[politica, eleicoes-2022, noticia]"
255599,http://g1.globo.com/politica/eleicoes/2022/not...,"[politica, eleicoes-2022, noticia]"
255600,http://g1.globo.com/politica/eleicoes/2022/not...,"[politica, eleicoes-2022, noticia]"
255601,http://g1.globo.com/politica/eleicoes/2022/not...,"[politica, eleicoes-2022, noticia]"


In [17]:
from collections import Counter

all_tags = [tag for tags_list in itens["tags"] for tag in tags_list]

df_tags = pd.DataFrame(Counter(all_tags).items(), columns=["tag", "frequency"])
df_tags = df_tags.sort_values(by="frequency", ascending=False)
df_tags

,tag,frequency
2,noticia,227921
99,sp,41364
48,mg,18319
70,rj,15094
71,rj-rio-de-janeiro,11287
...,...,...
2662,sp-enel-energia-legal,1
2661,uma-receita-de-amor,1
2660,integra-coopera,1
2659,coopera,1


Temos muitas tags e várias que aparecem apenas uma vez em nossa base. Por conta disso, optamos por processar as tags com embeddings - da mesma forma que outros textos - pois, dessa forma, até tags que aparecem pouco irão ter algum tipo de contexto ao se relacionar com outros campos (como o corpo da notícia).

In [18]:
df_tags.tail(50)

,tag,frequency
2585,tefara,1
2586,ekopress,1
2619,livelo,1
2620,casa-de-carnes-e-mercearia-marquinhos,1
2621,confiance,1
2622,manaus-o-futuro-e-agora,1
2623,prefeitura-de-irece,1
2624,governo-do-maranhao,1
2625,femec,1
2594,mti,1


Como vamos utilizar embeddings, gerar tags exclusivas para UF e cidade agora é desnecessário, já que elas serão relacionadas de qualquer forma durante o processo. Por conta disso, agora ao invés de gerar uma tag como `[pa, pa-santarem-regiao, noticia]`, agora teremos `[pa, santarem-regiao, noticia]`

In [19]:
def extract_tags(url: str) -> list[str]:
    """Extrai tags de uma URL."""
    url = re.sub(r"^https?://(especiais\.)?g1\.globo\.com/", "", url)

    parts = url.split("/")

    final_tags = []
    i = 0
    while i < len(parts):
        part = parts[i]

        if not part:
            i += 1
            continue

        if (
            re.fullmatch(r"\d{4}", part)
            and i + 2 < len(parts)
            and re.fullmatch(r"\d{2}", parts[i + 1])
            and re.fullmatch(r"\d{2}", parts[i + 2])
        ):
            # Encontrou uma data completa, para de processar
            break

        if re.search(r"\.(ghtml|html)$", part):
            # Encontrou o título, para de processar
            break

        # Se a parte atual não for um ano (4 dígitos)
        if not re.fullmatch(r"\d{4}", part):
            # Verifica se a próxima parte é um ano e não faz parte de uma data
            if (
                i + 1 < len(parts)
                and re.fullmatch(r"\d{4}", parts[i + 1])
                and not (
                    i + 3 < len(parts)
                    and re.fullmatch(r"\d{2}", parts[i + 2])
                    and re.fullmatch(r"\d{2}", parts[i + 3])
                )
            ):
                # Junta a parte atual com o ano (ex.: "eleicoes/2022" → "eleicoes-2022")
                final_tags.append(f"{part}-{parts[i + 1]}")
                i += 1  # Pula o ano
            else:
                # Adiciona a parte como uma tag normal
                final_tags.append(part)
        i += 1

    return list(dict.fromkeys(final_tags))

In [20]:
itens["tags"] = itens["url"].apply(extract_tags)
itens[["url", "tags"]]

,url,tags
0,http://g1.globo.com/am/amazonas/noticia/2022/0...,"[am, amazonas, noticia]"
1,http://g1.globo.com/pa/santarem-regiao/noticia...,"[pa, santarem-regiao, noticia]"
2,http://g1.globo.com/mundo/noticia/2022/07/08/e...,"[mundo, noticia]"
3,http://g1.globo.com/politica/noticia/2021/09/0...,"[politica, noticia]"
4,http://g1.globo.com/politica/noticia/2021/09/1...,"[politica, noticia]"
...,...,...
255598,http://g1.globo.com/politica/eleicoes/2022/not...,"[politica, eleicoes-2022, noticia]"
255599,http://g1.globo.com/politica/eleicoes/2022/not...,"[politica, eleicoes-2022, noticia]"
255600,http://g1.globo.com/politica/eleicoes/2022/not...,"[politica, eleicoes-2022, noticia]"
255601,http://g1.globo.com/politica/eleicoes/2022/not...,"[politica, eleicoes-2022, noticia]"


In [21]:
itens.head(1)

,page,url,issued,modified,title,body,caption,tags
0,13db0ab1-eea2-4603-84c4-f40a876c7400,http://g1.globo.com/am/amazonas/noticia/2022/0...,2022-06-18 20:37:45+00:00,2023-04-15 00:02:08+00:00,Caso Bruno e Dom: 3º suspeito tem prisão tempo...,"Após audiência de custódia, a Justiça do Amazo...",Jeferson da Silva Lima foi escoltado por agent...,"[am, amazonas, noticia]"


## Gerando popularidade

Além das tags, também é interessante gerar a popularidade de uma notícia. Com isso, poderemos recomendar notícias populares para usuários novos - enquanto o nosso modelo não for capaz de identificar os gostos do usuário, ele terá recomendações baseadas nas notícias mais populares.

In [22]:
train_path = "../data/raw/files/treino/*.csv"
train_files = Path.cwd().glob(train_path)

train = pd.concat(
    (read_csv_pyarrow(file) for file in train_files),
    ignore_index=True,
)
list_columns = [
    "history",
    "timestampHistory",
    "numberOfClicksHistory",
    "timeOnPageHistory",
    "scrollPercentageHistory",
    "pageVisitsCountHistory",
]

train = train.drop(columns=["timestampHistory_new"])
train[list_columns] = train[list_columns].apply(lambda x: x.str.split(", "))
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 577942 entries, 0 to 577941
Data columns (total 9 columns):
 #   Column                   Non-Null Count   Dtype                      
---  ------                   --------------   -----                      
 0   userId                   577942 non-null  string[pyarrow]            
 1   userType                 577942 non-null  string[pyarrow]            
 2   historySize              577942 non-null  int64[pyarrow]             
 3   history                  577942 non-null  list<item: string>[pyarrow]
 4   timestampHistory         577942 non-null  list<item: string>[pyarrow]
 5   numberOfClicksHistory    577942 non-null  list<item: string>[pyarrow]
 6   timeOnPageHistory        577942 non-null  list<item: string>[pyarrow]
 7   scrollPercentageHistory  577942 non-null  list<item: string>[pyarrow]
 8   pageVisitsCountHistory   577942 non-null  list<item: string>[pyarrow]
dtypes: int64[pyarrow](1), list<item: string>[pyarrow](6), string

In [23]:
validation_path = Path("../data/raw/validacao.csv")
validation = read_csv_pyarrow(validation_path)

list_columns = ["history", "timestampHistory"]

validation["history"] = (
    validation["history"]
    .str.replace(r"[\[\]']", "", regex=True)
    .str.replace(r"\n", "", regex=True)
    .str.split()
)
validation["timestampHistory"] = (
    validation["timestampHistory"].str.replace(r"[\[\]]", "", regex=True).str.split()
)
validation.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112184 entries, 0 to 112183
Data columns (total 4 columns):
 #   Column            Non-Null Count   Dtype                      
---  ------            --------------   -----                      
 0   userId            112184 non-null  string[pyarrow]            
 1   userType          112184 non-null  string[pyarrow]            
 2   history           112184 non-null  list<item: string>[pyarrow]
 3   timestampHistory  112184 non-null  list<item: string>[pyarrow]
dtypes: list<item: string>[pyarrow](2), string[pyarrow](2)
memory usage: 19.8 MB


In [24]:
def compute_popularity(itens: pd.DataFrame, users: pd.DataFrame) -> pd.DataFrame:
    """Compute all itens popularity based on users profile.

    Args:
        itens (DataFrame): dataframe with page info
        users (DataFrame): dataframe with users info

    """
    reference_date = pd.to_datetime(itens["issued"].max())
    itens["issued"] = pd.to_datetime(itens["issued"])
    itens["days_since_issue"] = (reference_date - itens["issued"]).dt.days

    popularity_dict = {}
    time_spent_dict = {}

    for _, row in users.iterrows():
        history = row["history"]
        time_on_page_history = row["timeOnPageHistory"]
        visits_count_history = row["pageVisitsCountHistory"]

        for news_id, time_spent, visits in zip(
            history, time_on_page_history, visits_count_history
        ):
            popularity_dict[news_id] = popularity_dict.get(news_id, 0) + 1
            time_spent_per_visit = pd.to_numeric(time_spent) / pd.to_numeric(visits)
            time_spent_dict.setdefault(news_id, []).append(
                # max time is five minutes on the page
                np.clip(time_spent_per_visit, 1, 300000)
            )

    popularity_df = pd.DataFrame(
        list(popularity_dict.items()), columns=["page", "unique_users"]
    )

    time_spent_median = {k: np.median(v) for k, v in time_spent_dict.items()}
    time_spent_df = pd.DataFrame(
        list(time_spent_median.items()), columns=["page", "median_time_on_page"]
    )

    decay_factor = 0.05

    itens = itens.merge(popularity_df, on="page", how="left").fillna(0)
    itens = itens.merge(time_spent_df, on="page", how="left").fillna(0)

    itens["log_unique_users"] = np.log1p(itens["unique_users"])

    itens["popularity_score_general"] = (itens["log_unique_users"] * 2) * (
        1 + 0.05 * itens["median_time_on_page"] / 1000
    )

    itens["popularity_score_decay"] = itens["popularity_score_general"] * np.exp(
        -decay_factor * itens["days_since_issue"]
    )

    return itens

In [25]:
# ruff: noqa: ERA001

# import os


# def compute_popularity2(itens, users, validations, output_file="popularity.parquet"):
#     validations["timestampHistory"] = validations["timestampHistory"].apply(
#         pd.to_numeric
#     )
#     users["timestampHistory"] = users["timestampHistory"].apply(
#         lambda x: list(map(int, x))
#     )

#     unique_timestamps = validations["timestampHistory"].explode().unique()
#     unique_timestamps = np.sort(
#         np.linspace(
#             unique_timestamps.min(),
#             unique_timestamps.max(),
#             num=12,
#             dtype=int,
#             endpoint=True,
#         )
#     )

#     display(f"Unique timestamps: {unique_timestamps}")

#     issued_dict = (
#         itens.set_index("page")["issued"]
#         .apply(lambda x: x.timestamp() * 1000)
#         .to_dict()
#     )

#     all_data = []
#     for _, row in users.iterrows():
#         for page_id, time_spent, timestamp in zip(
#             row["history"], row["timeOnPageHistory"], row["timestampHistory"]
#         ):
#             all_data.append((timestamp, page_id, time_spent))

#     users_expanded = pd.DataFrame(
#         all_data, columns=["timestamp", "page_id", "time_spent"]
#     )
#     users_expanded["timestamp"] = users_expanded["timestamp"].astype(int)

#     if os.path.exists(output_file):
#         processed_data = pd.read_parquet(output_file)
#         processed_timestamps = set(processed_data["timestamp"].unique())
#     else:
#         processed_timestamps = set()

#     for timestamp in unique_timestamps:
#         display(f"Gerando dados para o timestamp {timestamp}")
#         if timestamp in processed_timestamps:
#             continue

#         display("Filtrando users")
#         filtered_users = users_expanded[users_expanded["timestamp"] <= timestamp]

#         display("Agrupando")
#         grouped = (
#             filtered_users.groupby("page_id")
#             .agg(
#                 count=("page_id", "size"),
#                 median_time=(
#                     "time_spent",
#                     lambda x: np.median(x.astype(int).clip(upper=300000)),
#                 ),
#             )
#             .reset_index()
#         )

#         display("Calculando score")
#         grouped = grouped[grouped["page_id"].map(issued_dict) <= timestamp]
#         time_decay = np.exp(
#             -((timestamp - grouped["page_id"].map(issued_dict)) / (24 * 3600 * 1000))
#         )
#         grouped["popularity_score"] = (
#             np.log1p(grouped["count"] * 4 + grouped["median_time"]) * time_decay
#         )
#         grouped["timestamp"] = timestamp

#         if os.path.exists(output_file):
#             grouped[["timestamp", "page_id", "popularity_score"]].sort_values(
#                 "popularity_score", ascending=False
#             ).to_parquet(output_file, engine="fastparquet", append=True)
#         else:
#             grouped[["timestamp", "page_id", "popularity_score"]].sort_values(
#                 "popularity_score", ascending=False
#             ).to_parquet(output_file, engine="fastparquet")

# top_news_per_timestamp = compute_popularity2(itens=itens, users=train, validations=validation)  # noqa: E501

In [26]:
itens = compute_popularity(itens=itens, users=train)
itens

,page,url,issued,modified,title,body,caption,tags,days_since_issue,unique_users,median_time_on_page,log_unique_users,popularity_score_general,popularity_score_decay
0,13db0ab1-eea2-4603-84c4-f40a876c7400,http://g1.globo.com/am/amazonas/noticia/2022/0...,2022-06-18 20:37:45+00:00,2023-04-15 00:02:08+00:00,Caso Bruno e Dom: 3º suspeito tem prisão tempo...,"Após audiência de custódia, a Justiça do Amazo...",Jeferson da Silva Lima foi escoltado por agent...,"[am, amazonas, noticia]",57,3,10875.00,1.386294,4.280184,2.475843e-01
1,92907b73-5cd3-4184-8d8c-e206aed2bf1c,http://g1.globo.com/pa/santarem-regiao/noticia...,2019-06-20 17:19:52+00:00,2023-06-16 20:19:15+00:00,Linguajar dos santarenos é diferenciado e chei...,Vista aérea de Santarém Ádrio Denner/ AD Produ...,As expressões santarenas não significam apenas...,"[pa, santarem-regiao, noticia]",1151,2,15000.00,1.098612,3.845143,3.901801e-25
2,61e07f64-cddf-46f2-b50c-ea0a39c22050,http://g1.globo.com/mundo/noticia/2022/07/08/e...,2022-07-08 08:55:52+00:00,2023-04-15 04:25:39+00:00,Ex-premiê Shinzo Abe morre após ser baleado no...,Novo vídeo mostra que assassino de Shinzo Abe ...,Ex-primeiro-ministro foi atingido por tiros de...,"[mundo, noticia]",37,12498,60000.00,9.433404,75.467231,1.186625e+01
3,30e2e6c5-554a-48ed-a35f-6c6691c8ac9b,http://g1.globo.com/politica/noticia/2021/09/0...,2021-09-09 19:06:46+00:00,2023-06-07 17:44:54+00:00,"Relator no STF, Fachin vota contra marco tempo...","Relator no STF, Fachin vota contra marco tempo...",Ministro defendeu que posse indígena é diferen...,"[politica, noticia]",339,3,35856.50,1.386294,7.743355,3.370061e-07
4,9dff71eb-b681-40c7-ac8d-68017ac36675,http://g1.globo.com/politica/noticia/2021/09/1...,2021-09-15 19:16:13+00:00,2023-06-07 17:43:39+00:00,"Após 2 votos, pedido de vista suspende julgam...",Após um pedido de vista (mais tempo para análi...,"Pelo marco temporal, índios só podem reivindic...","[politica, noticia]",333,6,41051.25,1.945910,11.880025,6.979338e-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
255598,aca76b7b-9c02-4070-a4a9-00f8c9b4dae2,http://g1.globo.com/politica/eleicoes/2022/not...,2022-04-18 23:32:54+00:00,2022-05-02 10:53:35+00:00,"Eleições: título de eleitor cancelado, o que f...",Detalhe da urna eletrônica e a tecla confirma ...,Cidadão pode consultar a situação do título de...,"[politica, eleicoes-2022, noticia]",118,3,50000.00,1.386294,9.704061,2.658374e-02
255599,20de2172-b557-4881-8c55-2706db029e9b,http://g1.globo.com/politica/eleicoes/2022/not...,2022-08-09 00:53:27+00:00,2022-09-19 20:30:25+00:00,Plano de governo: Ciro Gomes (PDT),"Candidato do PDT à Presidência da República, C...","Em documento de 26 páginas, candidato à Presid...","[politica, eleicoes-2022, noticia]",6,155,47431.00,5.049856,34.051684,2.522611e+01
255600,347c7e3d-ff53-4ba0-8de7-8bb860b04a88,http://g1.globo.com/politica/eleicoes/2022/not...,2022-03-15 18:18:54+00:00,2022-03-15 18:26:34+00:00,Moro vê economia brasileira estagnada e diz qu...,O pré-candidato do Podemos à Presidência da Re...,Ex-ministro da Justiça de Bolsonaro e pré-cand...,"[politica, eleicoes-2022, noticia]",152,1,9409.00,0.693147,2.038477,1.020159e-03
255601,26f7ff70-46da-42d2-9354-578e8f828a16,http://g1.globo.com/politica/eleicoes/2022/not...,2022-03-23 15:13:50+00:00,2022-03-24 00:54:34+00:00,Lula é hoje 'aquele que melhor reflete o senti...,"Ex-governador de SP, Geraldo Alckmin se filia ...",Ex-governador de SP deu a declaração durante a...,"[politica, eleicoes-2022, noticia]",144,4,20027.25,1.609438,6.442137,4.809608e-03


In [27]:
(train["history"].explode() == "92907b73-5cd3-4184-8d8c-e206aed2bf1c").value_counts()

history
False    8123949
True           2
Name: count, dtype: int64[pyarrow]

In [28]:
with pd.option_context("display.float_format", "{:.3f}".format):
    display(
        itens  # .sort_values(by="popularity_score", ascending=False)
    )

,page,url,issued,modified,title,body,caption,tags,days_since_issue,unique_users,median_time_on_page,log_unique_users,popularity_score_general,popularity_score_decay
0,13db0ab1-eea2-4603-84c4-f40a876c7400,http://g1.globo.com/am/amazonas/noticia/2022/0...,2022-06-18 20:37:45+00:00,2023-04-15 00:02:08+00:00,Caso Bruno e Dom: 3º suspeito tem prisão tempo...,"Após audiência de custódia, a Justiça do Amazo...",Jeferson da Silva Lima foi escoltado por agent...,"[am, amazonas, noticia]",57,3,10875.000,1.386,4.280,0.248
1,92907b73-5cd3-4184-8d8c-e206aed2bf1c,http://g1.globo.com/pa/santarem-regiao/noticia...,2019-06-20 17:19:52+00:00,2023-06-16 20:19:15+00:00,Linguajar dos santarenos é diferenciado e chei...,Vista aérea de Santarém Ádrio Denner/ AD Produ...,As expressões santarenas não significam apenas...,"[pa, santarem-regiao, noticia]",1151,2,15000.000,1.099,3.845,0.000
2,61e07f64-cddf-46f2-b50c-ea0a39c22050,http://g1.globo.com/mundo/noticia/2022/07/08/e...,2022-07-08 08:55:52+00:00,2023-04-15 04:25:39+00:00,Ex-premiê Shinzo Abe morre após ser baleado no...,Novo vídeo mostra que assassino de Shinzo Abe ...,Ex-primeiro-ministro foi atingido por tiros de...,"[mundo, noticia]",37,12498,60000.000,9.433,75.467,11.866
3,30e2e6c5-554a-48ed-a35f-6c6691c8ac9b,http://g1.globo.com/politica/noticia/2021/09/0...,2021-09-09 19:06:46+00:00,2023-06-07 17:44:54+00:00,"Relator no STF, Fachin vota contra marco tempo...","Relator no STF, Fachin vota contra marco tempo...",Ministro defendeu que posse indígena é diferen...,"[politica, noticia]",339,3,35856.500,1.386,7.743,0.000
4,9dff71eb-b681-40c7-ac8d-68017ac36675,http://g1.globo.com/politica/noticia/2021/09/1...,2021-09-15 19:16:13+00:00,2023-06-07 17:43:39+00:00,"Após 2 votos, pedido de vista suspende julgam...",Após um pedido de vista (mais tempo para análi...,"Pelo marco temporal, índios só podem reivindic...","[politica, noticia]",333,6,41051.250,1.946,11.880,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
255598,aca76b7b-9c02-4070-a4a9-00f8c9b4dae2,http://g1.globo.com/politica/eleicoes/2022/not...,2022-04-18 23:32:54+00:00,2022-05-02 10:53:35+00:00,"Eleições: título de eleitor cancelado, o que f...",Detalhe da urna eletrônica e a tecla confirma ...,Cidadão pode consultar a situação do título de...,"[politica, eleicoes-2022, noticia]",118,3,50000.000,1.386,9.704,0.027
255599,20de2172-b557-4881-8c55-2706db029e9b,http://g1.globo.com/politica/eleicoes/2022/not...,2022-08-09 00:53:27+00:00,2022-09-19 20:30:25+00:00,Plano de governo: Ciro Gomes (PDT),"Candidato do PDT à Presidência da República, C...","Em documento de 26 páginas, candidato à Presid...","[politica, eleicoes-2022, noticia]",6,155,47431.000,5.050,34.052,25.226
255600,347c7e3d-ff53-4ba0-8de7-8bb860b04a88,http://g1.globo.com/politica/eleicoes/2022/not...,2022-03-15 18:18:54+00:00,2022-03-15 18:26:34+00:00,Moro vê economia brasileira estagnada e diz qu...,O pré-candidato do Podemos à Presidência da Re...,Ex-ministro da Justiça de Bolsonaro e pré-cand...,"[politica, eleicoes-2022, noticia]",152,1,9409.000,0.693,2.038,0.001
255601,26f7ff70-46da-42d2-9354-578e8f828a16,http://g1.globo.com/politica/eleicoes/2022/not...,2022-03-23 15:13:50+00:00,2022-03-24 00:54:34+00:00,Lula é hoje 'aquele que melhor reflete o senti...,"Ex-governador de SP, Geraldo Alckmin se filia ...",Ex-governador de SP deu a declaração durante a...,"[politica, eleicoes-2022, noticia]",144,4,20027.250,1.609,6.442,0.005


In [29]:
with pd.option_context("display.float_format", "{:.3f}".format):
    display(itens.sort_values(by="popularity_score_general", ascending=False))

,page,url,issued,modified,title,body,caption,tags,days_since_issue,unique_users,median_time_on_page,log_unique_users,popularity_score_general,popularity_score_decay
179633,0baa8cd7-1eef-4960-aeb4-4128e58efba3,http://g1.globo.com/sp/sao-paulo/noticia/2022/...,2022-07-25 11:23:02+00:00,2022-07-25 23:31:51+00:00,Cabeleireira é encontrada morta em casa ao lad...,Mulher é vítima de feminicídio em apartamento ...,Amigas e vizinhos estranharam sumiço da vítima...,"[sp, sao-paulo, noticia]",20,11198,70000.000,9.324,83.912,30.870
182055,5af379e6-1bd1-4cf8-a23c-03266fb77b2c,http://g1.globo.com/sp/sao-paulo/noticia/2022/...,2022-07-20 23:05:56+00:00,2022-07-21 14:44:12+00:00,Casa abandonada em Higienópolis: Entenda o cas...,Casa abandonada em SP: Entenda o caso da mulhe...,A casa e a história de Margarida Bonetti vêm c...,"[sp, sao-paulo, noticia]",25,11140,70000.000,9.318,83.865,24.028
26314,6a83890a-d9e9-4f6b-a6c6-90d031785bbf,http://g1.globo.com/pi/piaui/noticia/2022/07/2...,2022-07-27 13:54:29+00:00,2022-07-28 10:48:24+00:00,Pizzaria recebe PIX falso e entrega refrigeran...,Golpe do Pix: empresa entrega pizza e refriger...,"Com ajuda de funcionários, dono do estabelecim...","[pi, piaui, noticia]",18,18101,64845.000,9.804,83.180,33.819
42935,4e9c2825-ff13-41ca-8e91-edd848060d19,http://g1.globo.com/mg/minas-gerais/noticia/20...,2022-08-02 12:45:41+00:00,2022-08-03 11:08:58+00:00,Menina de 10 anos que estava desaparecida após...,O que se sabe sobre o caso da menina de 10 ano...,Corpo de Bárbara Vitória foi localizado em um ...,"[mg, minas-gerais, noticia]",12,14681,65822.000,9.594,82.341,45.190
209562,d57f0e9a-445b-465d-b757-be4f32e75fb9,http://g1.globo.com/ba/bahia/noticia/2022/07/2...,2022-07-20 22:03:43+00:00,2022-07-21 20:32:17+00:00,Mulher relata assédio de urologista na BA; men...,'Não posso me calar': paciente diz que urologi...,A paciente também afirmou que foi beijada na t...,"[ba, bahia, noticia]",25,9382,70000.000,9.147,82.320,23.585
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48423,af0583dc-16ba-4edd-80a1-59c12862136a,http://g1.globo.com/ms/mato-grosso-do-sul/noti...,2022-01-03 16:04:48+00:00,2022-01-03 16:59:04+00:00,Governo de MS decreta situação de emergência d...,O governo de Mato Grosso do Sul decretou situa...,Medida vale para os 79 municípios de Mato Gros...,"[ms, mato-grosso-do-sul, noticia]",223,1,5008.000,0.693,1.733,0.000
86598,d80131a4-5d08-4378-a384-28c15b52fa11,http://g1.globo.com/sp/sao-carlos-regiao/notic...,2019-05-28 22:20:25+00:00,2019-05-28 23:01:29+00:00,Tio é condenado a 40 anos de prisão por matar ...,Hemilly Brenda Gonçalves de Oliveira morreu ap...,Adolescente foi espancada quando disse que tin...,"[sp, sao-carlos-regiao, noticia]",1174,1,5003.000,0.693,1.733,0.000
124602,748301cc-be7e-498e-afcf-2068c931aa18,http://g1.globo.com/pr/parana/concursos-e-empr...,2022-05-04 23:05:57+00:00,2022-05-04 23:35:40+00:00,Multinacional abre inscrições para vagas de es...,Carteira de trabalho Divulgação/Prefeitura de...,Vagas têm bolsa-auxílio a partir de R$ 1.350 p...,"[pr, parana, concursos-e-emprego, noticia]",102,1,5003.000,0.693,1.733,0.011
97490,5995e8f9-1afd-47c7-89ce-43a405b2e6cf,http://g1.globo.com/ro/rondonia/noticia/2022/0...,2022-02-10 23:46:49+00:00,2022-02-10 23:46:50+00:00,Justiça determina que criança de RO levada ile...,Quatro meses depois de denunciar ao g1 o seque...,Guarda provisória da criança foi concedida ao ...,"[ro, rondonia, noticia]",185,1,5002.000,0.693,1.733,0.000


In [30]:
with pd.option_context("display.float_format", "{:.3f}".format):
    display(itens.sort_values(by="popularity_score_decay", ascending=False))

,page,url,issued,modified,title,body,caption,tags,days_since_issue,unique_users,median_time_on_page,log_unique_users,popularity_score_general,popularity_score_decay
71096,d730c4a6-e8f6-4fde-b73a-afbe148479cd,http://g1.globo.com/sp/santos-regiao/noticia/2...,2022-08-14 09:49:40+00:00,2022-08-15 12:41:00+00:00,Mulher é vítima de racismo em shopping no lito...,Vítima relata ofensas racistas em shopping no ...,Filha da vítima contou ao g1 que a mãe está em...,"[sp, santos-regiao, noticia]",0,4533,68569.000,8.419,74.569,74.569
171665,1f2b9c2f-a2d2-4192-b009-09065da8ec23,http://g1.globo.com/rj/rio-de-janeiro/noticia/...,2022-08-12 09:49:53+00:00,2022-08-12 13:07:13+00:00,"VÍDEO: ‘Me ajuda, por favor! Não! Socorro!’, s...","'Me ajuda, por favor!', suplica influenciadora...","Namorado afirmava que, se ela fugisse, ele iri...","[rj, rio-de-janeiro, noticia]",2,9744,66110.000,9.185,79.088,71.562
99533,598ed114-fd5a-4d82-90d8-f1e893cb0892,http://g1.globo.com/sp/santos-regiao/noticia/2...,2022-08-11 07:51:13+00:00,2022-08-11 19:16:38+00:00,Mãe descobre paradeiro de filha desaparecida p...,"Karla (à esq) e Marcela, irmãs vão se reencont...","Mulher que mora no Morro da Nova Cintra, em Sa...","[sp, santos-regiao, noticia]",3,8449,70000.000,9.042,81.377,70.042
120511,d173164f-dcfc-4e32-8006-64ab44a57c66,http://g1.globo.com/rj/rio-de-janeiro/noticia/...,2022-08-11 14:04:19+00:00,2022-08-15 19:13:50+00:00,"Golpe em idosa: ‘Mata essa velha!’, mandou Ro...",Sabine Boghici Reprodução Rosa Stanesco Nicola...,As namoradas e outras duas pessoas foram presa...,"[rj, rio-de-janeiro, noticia]",3,7318,68426.500,8.898,78.684,67.724
205507,279ccbff-f203-4c6d-aa48-83d97f085302,http://g1.globo.com/sp/sao-paulo/noticia/2022/...,2022-08-13 22:51:15+00:00,2022-08-15 01:05:01+00:00,Policial que matou lutador Leandro Lo foi a bo...,Policial que matou lutador Leandro Lo foi a bo...,Henrique Veloso saiu de boate com garota de pr...,"[sp, sao-paulo, noticia]",1,7314,60000.000,8.898,71.181,67.710
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63939,esid:conteudo_editorial_g1#materia#http://espe...,http://especiais.g1.globo.com/minas-gerais/201...,2015-06-30 07:00:00+00:00,2018-12-28 19:42:21+00:00,As promessas de Pimentel,Pimentel fez promessas específicas em um progr...,Pimentel fez promessas específicas em um progr...,"[minas-gerais-2015, as-promessas-de-pimentel]",2602,11,12670.500,2.485,8.118,0.000
207396,esid:conteudo_editorial_g1#materia#https://esp...,http://especiais.g1.globo.com/minas-gerais/201...,2015-06-30 07:00:00+00:00,2022-12-06 11:02:06+00:00,As promessas de Pimentel,Pimentel fez promessas específicas em um progr...,Pimentel fez promessas específicas em um progr...,"[minas-gerais-2015, as-promessas-de-pimentel]",2602,1,36987.000,0.693,3.950,0.000
207401,esid:conteudo_editorial_g1#materia#https://esp...,http://especiais.g1.globo.com/distrito-federal...,2015-06-30 07:00:00+00:00,2022-12-06 11:04:01+00:00,As promessas de Rollemberg,Rollemberg fez promessas específicas em um pro...,Rollemberg fez promessas específicas em um pro...,"[distrito-federal-2015, as-promessas-de-rollem...",2602,1,10000.000,0.693,2.079,0.000
208312,esid:conteudo_editorial_g1#materia#http://espe...,http://especiais.g1.globo.com/parana/2015/as-p...,2015-01-12 07:00:00+00:00,2018-12-30 16:04:56+00:00,As promessas de Richa,Richa fez promessas específicas em um programa...,Richa fez promessas específicas em um programa...,"[parana-2015, as-promessas-de-richa]",2771,11,40000.000,2.485,14.909,0.000


In [31]:
itens.to_parquet("../data/interim/itens_popularity.parquet", engine="pyarrow")

<!--  -->